In [1]:
import pandas as pd
import numpy as np
import datetime 

data = pd.read_csv(r'C:\Users\nadia\Dropbox\Internship - Kairaktidi\data-ypen\ΠΕΙΡΑΙΑΣ-I-PIR\PIRPYTHON.csv')

print(data)

            DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3   CO
0      01-01-20 0:00   9.98  57.0  8.0  133.0  112.0   8.0  NaN
1      01-01-20 1:00  11.15  56.0  8.0  142.0  120.0  10.0  NaN
2      01-01-20 2:00  11.71  53.0  9.0  197.0  175.0   6.0  NaN
3      01-01-20 3:00   9.26  46.0  7.0  146.0  123.0   6.0  NaN
4      01-01-20 4:00   9.68  45.0  8.0  133.0  117.0   5.0  NaN
...              ...    ...   ...  ...    ...    ...   ...  ...
8779  31-12-20 19:00   0.77  32.0  4.0   44.0   12.0  40.0  1.6
8780  31-12-20 20:00   1.15  52.0  5.0   46.0   16.0  26.0  2.2
8781  31-12-20 21:00   0.61  36.0  5.0   47.0   21.0  43.0  2.4
8782  31-12-20 22:00   0.40  52.0  4.0   25.0    6.0  28.0  1.6
8783  31-12-20 23:00   0.48  41.0  6.0   27.0    8.0  38.0  0.9

[8784 rows x 8 columns]


In [25]:
data['DATETIME']=pd.to_datetime(data['DATETIME'])
data.groupby(by=data['DATETIME'].dt.date).count()
#data.groupby(by=data['DATETIME'].dt.date).mean()

data['O3roll'] = data['O3'].rolling(8).mean()
data['COroll'] = data['CO'].rolling(8).mean()

#print(data['COroll'].head(50))

In [3]:
data['date'] = data['DATETIME'].dt.date
print(data)

                DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3   CO  O3roll  \
0    2020-01-01 00:00:00   9.98  57.0  8.0  133.0  112.0   8.0  NaN     NaN   
1    2020-01-01 01:00:00  11.15  56.0  8.0  142.0  120.0  10.0  NaN     NaN   
2    2020-01-01 02:00:00  11.71  53.0  9.0  197.0  175.0   6.0  NaN     NaN   
3    2020-01-01 03:00:00   9.26  46.0  7.0  146.0  123.0   6.0  NaN     NaN   
4    2020-01-01 04:00:00   9.68  45.0  8.0  133.0  117.0   5.0  NaN     NaN   
...                  ...    ...   ...  ...    ...    ...   ...  ...     ...   
8779 2020-12-31 19:00:00   0.77  32.0  4.0   44.0   12.0  40.0  1.6  45.250   
8780 2020-12-31 20:00:00   1.15  52.0  5.0   46.0   16.0  26.0  2.2  43.750   
8781 2020-12-31 21:00:00   0.61  36.0  5.0   47.0   21.0  43.0  2.4  43.250   
8782 2020-12-31 22:00:00   0.40  52.0  4.0   25.0    6.0  28.0  1.6  40.875   
8783 2020-12-31 23:00:00   0.48  41.0  6.0   27.0    8.0  38.0  0.9  39.750   

      COroll        date  
0        NaN  2020-01-01

# C6H6

In [26]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας

data.replace('', np.nan).count()
c = data['C6H6'].count()
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", c)
if c > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 7029
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [5]:
# Έλεγχος οριακής τιμής στο ημερολογιακό έτος (5 μg/m3)

print("Μέση παρατηρούμενη ετήσια τιμή:", round(data['C6H6'].mean(), 2), "μg/m3")

if data['C6H6'].mean() > 5:
    print("Γίνεται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)")
    

Μέση παρατηρούμενη ετήσια τιμή: 2.04 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)


# NO2

In [27]:
# Ωριαία οριακή τιμή (200μg/m3 έως 18 φορές τον χρόνο)

print("Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών")

if data.loc[data.NO2 < 200, 'NO2'].count() > 18 :
     print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος")
else:
    print("Γίνεται υπέρβαση του ορίου")

Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών
Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος


In [28]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ετήσια οριακή τιμή 

#print(sum(pd.isnull(data['NO2'])))

n = data['NO2'].count()
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", n)
if n > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 8765
Ικανοποιείται ο έλεγχος πληρότητας


In [29]:
# Οριακή τιμή στο ημερολογ. έτος = 40 μg/m3

if data['NO2'].mean() > 40:
    print("Γίνεται υπέρβαση του ορίου,", round(data['NO2'].mean(), 1), "μg/m3 με όριο τα 40 μg/m3")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου ( 40 μg/m3)")

Γίνεται υπέρβαση του ορίου, 55.0 μg/m3 με όριο τα 40 μg/m3


# SO2

In [9]:
# Ωριαία οριακή τιμή (350μg/m3 έως 24 φορές τον χρόνο)

print("Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών","\n")

if data.loc[data.SO2 < 350, 'NO2'].count() > 24 :
     print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος")
else:
    print("Γίνεται υπέρβαση του ορίου")

print("Οι συνολικές υπερβάσεις ήταν:" ,data.loc[data.SO2 > 350, 'NO2'].count())

Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών 

Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος
Οι συνολικές υπερβάσεις ήταν: 0


In [10]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ημερήσια οριακή τιμή

grouped_single = data.groupby('date').agg({'SO2': ['count','max'], 'PM10': ['count','max','mean'], 'PM2.5': ['count','max','mean'], 'O3roll': ['count','max'] , 'COroll': ['count','max']})
grouped_single.replace('', np.nan).count()

s = grouped_single['SO2']['count'].le(18) #άκυρες και μέρες με λιγότερο από 18 ώρες μαζί

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['SO2']) - s.sum())
    
grouped_single.head(75)

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 354


SO2        PM10                   PM2.5                   O3roll  \
           count   max count    max       mean count    max       mean  count   
date                                                                            
2020-01-01    24  14.0    24  197.0  63.500000    24  175.0  49.375000     17   
2020-01-02    24  24.0    24  102.0  32.916667    24   56.0  15.416667     24   
2020-01-03    24  20.0    24   54.0  28.583333    11   38.0  23.545455     24   
2020-01-04    24   7.0    24   51.0  43.333333    24   27.0  20.500000     24   
2020-01-05    24   9.0    24   27.0  17.500000    24   13.0   8.250000     24   
...          ...   ...   ...    ...        ...   ...    ...        ...    ...   
2020-03-11    24  17.0    24   87.0  50.583333    24   38.0  25.541667     24   
2020-03-12    24  12.0    24   50.0  28.375000    24   32.0  15.791667     24   
2020-03-13    24  13.0    24   29.0  18.208333    22   14.0   8.272727     24   
2020-03-14    24  31.0    24   66.0  33.541667    24   37.0  18.833333     24   
2020-03-15     0   NaN    24   68.0  34.458333    24   34.0  20.083333     24   

                   COroll          
               max  count     max  
date                               
2020-01-01  29.125      0     NaN  
2020-01-02  73.625     24  2.6500  
2020-01-03  74.875     24  1.3500  
2020-01-04  84.500     24  0.4000  
2020-01-05  81.375     24  0.4625  
...            ...    ...     ...  
2020-03-11   8.750     24  3.5375  
2020-03-12  28.000     24  0.6750  
2020-03-13  46.625     24  2.6375  
2020-03-14  55.500     24  3.1500  
2020-03-15  55.875     24  1.0000  

[75 rows x 12 columns]

In [11]:
# Ημερήσια οριακή τιμή (125μg/m3 έως 3 φορές τον χρόνο)

s2 = grouped_single['SO2']['max'].gt(125)
if s2.sum() > 3:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", s2.sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 0


# PM10

In [12]:
pm = grouped_single['PM10']['count'].le(18)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['PM10']) - pm.sum())

grouped_single.head(75)

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 321


SO2        PM10                   PM2.5                   O3roll  \
           count   max count    max       mean count    max       mean  count   
date                                                                            
2020-01-01    24  14.0    24  197.0  63.500000    24  175.0  49.375000     17   
2020-01-02    24  24.0    24  102.0  32.916667    24   56.0  15.416667     24   
2020-01-03    24  20.0    24   54.0  28.583333    11   38.0  23.545455     24   
2020-01-04    24   7.0    24   51.0  43.333333    24   27.0  20.500000     24   
2020-01-05    24   9.0    24   27.0  17.500000    24   13.0   8.250000     24   
...          ...   ...   ...    ...        ...   ...    ...        ...    ...   
2020-03-11    24  17.0    24   87.0  50.583333    24   38.0  25.541667     24   
2020-03-12    24  12.0    24   50.0  28.375000    24   32.0  15.791667     24   
2020-03-13    24  13.0    24   29.0  18.208333    22   14.0   8.272727     24   
2020-03-14    24  31.0    24   66.0  33.541667    24   37.0  18.833333     24   
2020-03-15     0   NaN    24   68.0  34.458333    24   34.0  20.083333     24   

                   COroll          
               max  count     max  
date                               
2020-01-01  29.125      0     NaN  
2020-01-02  73.625     24  2.6500  
2020-01-03  74.875     24  1.3500  
2020-01-04  84.500     24  0.4000  
2020-01-05  81.375     24  0.4625  
...            ...    ...     ...  
2020-03-11   8.750     24  3.5375  
2020-03-12  28.000     24  0.6750  
2020-03-13  46.625     24  2.6375  
2020-03-14  55.500     24  3.1500  
2020-03-15  55.875     24  1.0000  

[75 rows x 12 columns]

In [13]:
# Οριακή τιμή ημέρας = 50μg/m3 έως 35 φορές το έτος

pm2 = grouped_single['PM10']['mean'].gt(50) #(εδώ θέλω την μέση τιμή της μέρας ή την μέγιστη;)
if pm2.sum() > 35:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", pm2.sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 27


In [14]:
# Έλεγχος πληρότητας για το έτος 

print("Συνολικές τιμές:", grouped_single.shape[0])

pm3 = grouped_single['PM10']['mean'].count()

print("Οι έγκυρες τιμές είναι", pm3)

if pm3 > (0.9*grouped_single.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Συνολικές τιμές: 366
Οι έγκυρες τιμές είναι 329
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [15]:
# Οριακή τιμή έτους = 40 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

print("Η οριακή τιμή του έτους είναι", round(grouped_single['PM10']['mean'].mean(),1), "μg/m3")

if grouped_single['PM10']['mean'].mean() > 40:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")

Η οριακή τιμή του έτους είναι 29.8 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου


# PM2.5

In [16]:
# Έλεγχος πληρότητας για το έτος 

print("Συνολικές τιμές:", grouped_single.shape[0])

pm5 = grouped_single['PM2.5']['count'].le(18) 

print("Οι έγκυρες τιμές είναι", len(grouped_single['PM10']) - pm5.sum())

if len(grouped_single['PM10']) - pm5.sum() > (0.9*grouped_single.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Συνολικές τιμές: 366
Οι έγκυρες τιμές είναι 333
Ικανοποιείται ο έλεγχος πληρότητας


In [17]:
# Οριακή τιμή έτους = 25 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

print("Η οριακή τιμή του έτους είναι", round(grouped_single['PM2.5']['mean'].mean(),1), "μg/m3")

if grouped_single['PM2.5']['mean'].mean() > 40:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")

Η οριακή τιμή του έτους είναι 14.6 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου


# O3

In [18]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

o = grouped_single['O3roll']['count'].le(18)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['O3roll']) - o.sum())

print(o.sum())

pd.set_option('display.max_rows', 366)
print(grouped_single['O3roll']['count'])

# different results from excel 

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 357
9
date
2020-01-01    17
2020-01-02    24
2020-01-03    24
2020-01-04    24
2020-01-05    24
2020-01-06    24
2020-01-07    24
2020-01-08    24
2020-01-09    24
2020-01-10    24
2020-01-11    24
2020-01-12    24
2020-01-13    24
2020-01-14    24
2020-01-15    13
2020-01-16    24
2020-01-17    24
2020-01-18    24
2020-01-19    24
2020-01-20    24
2020-01-21    24
2020-01-22    24
2020-01-23    24
2020-01-24    24
2020-01-25    24
2020-01-26    24
2020-01-27    24
2020-01-28    24
2020-01-29    24
2020-01-30    24
2020-01-31    24
2020-02-01    24
2020-02-02    24
2020-02-03    24
2020-02-04    24
2020-02-05    24
2020-02-06    24
2020-02-07    24
2020-02-08    24
2020-02-09    24
2020-02-10    24
2020-02-11    24
2020-02-12    24
2020-02-13    24
2020-02-14    24
2020-02-15    24
2020-02-16    24
2020-02-17    24
2020-02-18    24
2020-02-19    24
2020-02-20    24
2020-02-21    24
2020-02-22    24
2020-02-23    24
2020-02-24    24
20

In [19]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 120μg/m3 έως 25 φορές τον χρόνο σε διάστημα 3 χρόνων

print("Η οριακή τιμή του έτους είναι", round(grouped_single['O3roll']['max'].max(), 1), "μg/m3")

if grouped_single['O3roll']['max'].max() > 120:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές παραβιάσεις:", grouped_single['O3roll']['max'].gt(120).sum() )

Η οριακή τιμή του έτους είναι 108.9 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές παραβιάσεις: 0


# CO

the data for this pollutant are not for the station of Pireus but for the station of Patision, Greece

In [22]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

c = grouped_single['COroll']['count'].le(18)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['COroll']) - c.sum())

print(c.sum())
pd.set_option('display.max_rows', 50)
print(grouped_single['COroll']['count'])

# different results from excel 

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 332
34
date
2020-01-01     0
2020-01-02    24
2020-01-03    24
2020-01-04    24
2020-01-05    24
              ..
2020-12-27    24
2020-12-28    24
2020-12-29    24
2020-12-30    24
2020-12-31    24
Name: count, Length: 366, dtype: int64


In [30]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 10mg/m3 

print("Η οριακή τιμή του έτους είναι", round(grouped_single['COroll']['max'].max(), 2), "mg/m3")

if grouped_single['COroll']['max'].max() > 10:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές παραβιάσεις:", grouped_single['COroll']['max'].gt(10).sum() )

Η οριακή τιμή του έτους είναι 5.27 mg/m3
Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές παραβιάσεις: 0


χρειάζεται διόρθωση στον έλεγχο πληρότητας για το όζον και το μονοξείδιο του άνθρακα